In [1]:
"""This script estimates TRFs for several models and saves them"""
from pathlib import Path
import re
import eelbrain
import mne


DATA_ROOT = Path("~").expanduser() / 'Data' / 'cocoha'

ENVELOPES_DIR = DATA_ROOT / 'envelopes'
EEG_DIR = DATA_ROOT / 'eeg'

TRF_DIR = DATA_ROOT / 'TRFs'
TRF_DIR.mkdir(exist_ok=True)

SUBJECTS = [path.name for path in EEG_DIR.iterdir() if re.match(r'S\d+', path.name)]

In [2]:
# Loop through subjects to estimate TRFs
for SUBJECT in SUBJECTS:
    print("-" * 50)
    print(f"Processing subject {SUBJECT}...")
    
    # Load EEG data
    eeg_path = EEG_DIR / SUBJECT / f"{SUBJECT}_raw.fif"
    raw = mne.io.read_raw_fif(eeg_path)
    eeg = eelbrain.load.mne.raw_ndvar(raw)

    # Load envelope data
    attended_envelope_path = ENVELOPES_DIR / SUBJECT / f"{SUBJECT}_attended_envelope.pickle"
    unattended_envelope_path = ENVELOPES_DIR / SUBJECT / f"{SUBJECT}_unattended_envelope.pickle"
    attended_envelope = eelbrain.load.unpickle(attended_envelope_path)
    unattended_envelope = eelbrain.load.unpickle(unattended_envelope_path)

    print(f"EEG: {eeg}")
    print(f"Attended envelope: {attended_envelope}")
    print(f"Unattended envelope: {unattended_envelope}")

    # Estimate TRFs for attended and unattended conditions using boosting
    trf_attended = eelbrain.boosting(
            eeg,
            attended_envelope,
            -0.100,
            1.000,
            error='l1',
            basis=0.050,
            partitions=5,
            test=1,
            selective_stopping=True
        )
    
    trf_unattended = eelbrain.boosting(
            eeg,
            unattended_envelope,
            -0.100,
            1.000,
            error='l1',
            basis=0.050,
            partitions=5,
            test=1,
            selective_stopping=True
        )
    

    # Save TRFs
    trf_attended_path = TRF_DIR / SUBJECT / f"{SUBJECT}_attended_trf.pickle"
    trf_unattended_path = TRF_DIR / SUBJECT / f"{SUBJECT}_unattended_trf.pickle"
    trf_attended_path.parent.mkdir(exist_ok=True, parents=True)
    trf_unattended_path.parent.mkdir(exist_ok=True, parents=True)
    
    eelbrain.save.pickle(trf_attended, trf_attended_path)
    eelbrain.save.pickle(trf_unattended, trf_unattended_path)

    print(f"Saved attended TRF to {trf_attended_path}")
    print(f"Saved unattended TRF to {trf_unattended_path}")

--------------------------------------------------
Processing subject S5...
EEG: <NDVar 'S5_raw.fif': 64 sensor, 192000 time>
Attended envelope: <NDVar 'attended': 192000 time>
Unattended envelope: <NDVar 'unattended': 192000 time>
Saved attended TRF to /Users/sylvestereley/Data/cocoha/TRFs/S5/S5_attended_trf.pickle
Saved unattended TRF to /Users/sylvestereley/Data/cocoha/TRFs/S5/S5_unattended_trf.pickle
--------------------------------------------------
Processing subject S2...
EEG: <NDVar 'S2_raw.fif': 64 sensor, 192000 time>
Attended envelope: <NDVar 'attended': 192000 time>
Unattended envelope: <NDVar 'unattended': 192000 time>
Saved attended TRF to /Users/sylvestereley/Data/cocoha/TRFs/S2/S2_attended_trf.pickle
Saved unattended TRF to /Users/sylvestereley/Data/cocoha/TRFs/S2/S2_unattended_trf.pickle
--------------------------------------------------
Processing subject S3...
EEG: <NDVar 'S3_raw.fif': 64 sensor, 192000 time>
Attended envelope: <NDVar 'attended': 192000 time>
Unatten